In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

In [ ]:
from greybox_generalize import 

In [7]:
class LogDetModel(ExternalGreyBoxModel):
    def __init__(self, use_exact_derivatives=True,verbose=True):
        self._use_exact_derivatives = use_exact_derivatives
        self.verbose = verbose

        # For use with exact Hessian
        self._output_con_mult_values = np.zeros(1)

        if not use_exact_derivatives:
            raise NotImplementedError("use_exact_derivatives == False not supported")
        
    def input_names(self):
        """List of names, same order as in set_input_values
        """
        return ['a', 'b', 'c']

    def equality_constraint_names(self):
        """String names corresponding to any residuals for this external model.
        """
        return [ ]
    
    def output_names(self):
        return ['log_det']

    def set_output_constraint_multipliers(self, output_con_multiplier_values):
        assert len(output_con_multiplier_values) == 1
        np.copyto(self._output_con_mult_values, output_con_multiplier_values)

    def finalize_block_construction(self, pyomo_block):
        # set lower bounds on the variables
        #pyomo_block.inputs['a'].setlb(-10)
        #pyomo_block.inputs['b'].setlb(-10)
        #pyomo_block.inputs['c'].setlb(-10)

        # set upper bounds on the variables
        #pyomo_block.inputs['a'].setub(10)
        #pyomo_block.inputs['b'].setub(10)
        #pyomo_block.inputs['c'].setub(10)

        # initialize the variables
        pyomo_block.inputs['a'].value = 1
        pyomo_block.inputs['b'].value = 0
        pyomo_block.inputs['c'].value = 1
        
        #pyomo_block.add_const1 = pyo.Constraint(rule=pyomo_block.inputs['a']==self.initial[0])
        #pyomo_block.add_const2 = pyo.Constraint(rule=pyomo_block.inputs['b']==self.initial[1])
        #pyomo_block.add_const3 = pyo.Constraint(rule=pyomo_block.inputs['c']==self.initial[2])

    def set_input_values(self, input_values):
        self._input_values = list(input_values)
        
    def evaluate_equality_constraints(self):
        """Compute the residuals from the moodel using values set in input_values, return as a np array
        """
        return None
    
    def evaluate_outputs(self):
        """Compute outputs from the model using values set in input_values, return as a numpy array
        """
        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]

        # form matrix
        M = np.array([[a,b],[b,c]])

        # compute log determinant
        (sign, logdet) = np.linalg.slogdet(M)

        if self.verbose:
            print("\n Consider M =\n",M)
            print("   logdet = ",logdet,"\n")

        return np.asarray([logdet], dtype=np.float64)

    def evaluate_jacobian_equality_constraints(self):
        return None

    def evaluate_jacobian_outputs(self):
        """compute the derivatives of the ooutputs with respect to thee inputs 
        Return a scipy matrix with the rows in the order of the output variables,
        cols in the order of the input variables
        """

        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]

        if self._use_exact_derivatives:

            M = np.array([[a,b],[b,c]])

            # compute inverse
            Minv = np.linalg.pinv(M)

            row = np.zeros(3)
            col = np.zeros(3)
            data = np.zeros(3)
            row[0], col[0], data[0] = (0, 0, Minv[0,0])
            row[1], col[1], data[1] = (0, 1, 2*Minv[0,1])
            row[2], col[2], data[2] = (0, 2, Minv[1,1])
            return coo_matrix((data, (row, col)), shape=(1,3))


In [8]:
def build_model_external(m):
    ex_model = LogDetModel()
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

In [9]:
fim_prior = np.identity((2))
print(fim_prior)

[[1. 0.]
 [0. 1.]]


In [16]:
def compute_FIM():
    
    m = pyo.ConcreteModel()
    
    m.DimFIM = pyo.Set(initialize=[0,1])
    
    m.q = pyo.Var(m.DimFIM, initialize={0:2, 1:5})
    
    # set up y and cov y 
    def identity(m, a, b):
        return 1 if a==b else 0
    m.FIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=identity)
    
    def qtoFIM(m, a, b):
        return m.FIM[a,a]
    
    def symmetry(m, a, b):
        if a > b:
            return m.FIM[a,b]==m.FIM[b,a]
        #else:
        #    return pyo.Constraint.Skip
        elif a==b:
            return m.FIM[a,b]==2*m.q[a]+m.q[b]+20
        else:
            return pyo.Constraint.Skip
        
    m.con1 = pyo.Constraint(m.DimFIM, m.DimFIM, rule=symmetry)
    
    #fim_values = ['FIM[0,0]', 'FIM[0,1]', 'FIM[1,1]']

    # create a block to store the external grey box model
    #m.my_block = ExternalGreyBoxBlock(external_model=LogDetModel(fim_values))
    def _model_i(b):
        build_model_external(b)
    m.my_block = pyo.Block(rule=_model_i)
        
    def eq_fim00(m):
        return m.FIM[0,0] == m.my_block.egb.inputs['a']
    
    def eq_fim01(m):
        return m.FIM[0,1] == m.my_block.egb.inputs['b']
                  
    def eq_fim11(m):
        return m.FIM[1,1] == m.my_block.egb.inputs['c']
                  
    m.eq1 = pyo.Constraint(rule=eq_fim00)
    m.eq2 = pyo.Constraint(rule=eq_fim01)
    m.eq3 = pyo.Constraint(rule=eq_fim11)

    # add objective to maximize log det
    m.obj = pyo.Objective(expr=m.my_block.egb.outputs['log_det'], sense=pyo.maximize)

    return m    

In [17]:
mod = compute_FIM()

solver = pyo.SolverFactory('cyipopt')

solver.config.options['hessian_approximation'] = 'limited-memory' 

additional_options={'max_iter':3000}
for k,v in additional_options.items():
    solver.config.options[k] = v

results = solver.solve(mod, tee=True)


 Consider M =
 [[1. 0.]
 [0. 1.]]
   logdet =  0.0 


 Consider M =
 [[1.80369844 0.        ]
 [0.         2.27738265]]
   logdet =  1.412866064372556 


 Consider M =
 [[2.15788466 0.        ]
 [0.         2.55789991]]
   logdet =  1.7083149910962556 


 Consider M =
 [[2.9826002  0.        ]
 [0.         3.68299963]]
   logdet =  2.3965230085947797 


 Consider M =
 [[5.52290312 0.        ]
 [0.         6.71154996]]
   logdet =  3.6127335666884584 


 Consider M =
 [[ 8.36290836  0.        ]
 [ 0.         10.3285373 ]]
   logdet =  4.458716932252251 


 Consider M =
 [[13.7740413   0.        ]
 [ 0.         16.94358059]]
   logdet =  5.45267479154581 


 Consider M =
 [[22.09184722  0.        ]
 [ 0.         27.30449721]]
   logdet =  6.402260057976007 


 Consider M =
 [[35.79783482  0.        ]
 [ 0.         44.26705392]]
   logdet =  7.368128108067115 


 Consider M =
 [[57.89967823  0.        ]
 [ 0.         71.52649049]]
   logdet =  8.328779704690202 


 Consider M =
 [[ 93.69

In [19]:
### FIM
dim = 2 
fim_result = np.zeros((dim,dim))
for i in range(dim):
    for j in range(dim):
        fim_result[i,j] = pyo.value(mod.FIM[i,j])
        
print(pyo.value(mod.q[0]), pyo.value(mod.q[1]))
print(fim_result)  
print('log det from Pyomo FIM:', np.log(np.linalg.det(fim_result)))
print("Loge_Det from Pyomo OF:", pyo.value(mod.obj))

184689202837.20602 228157027417.25598
[[5.54067609e+11 0.00000000e+00]
 [0.00000000e+00 6.84471082e+11]]
log det from Pyomo FIM: 54.29246478759245
Loge_Det from Pyomo OF: 54.29246478759245


In [ ]:
mod.pprint()